## Strategy for counting fingers:
* Grab an ROI
* Calulate a running avergae background value for 60 frames of the video 
* Once the mean value is found, then the hand can enter the ROI  

Once the hand enters the ROI, we will use a Convex Hull to draw a polygon around the hand

We'll calculate the center of the hand against the angle of the outer points to infer finger count

In [1]:
import cv2 
import numpy as np
from sklearn.metrics import pairwise

%config Completer.use_jedi = False
%env OPENCV_VIDEOIO_PRIORITY_MSMF 0

env: OPENCV_VIDEOIO_PRIORITY_MSMF=0


In [2]:
background = None
accumlated_weight = 0.5

roi_top = 30
roi_bottom = 300
roi_right= 300
roi_left = 600

In [3]:
def calc_accum_avg(frame,accumlated_weight):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame, background,accumlated_weight)

In [4]:
def segment(frame,threshold=25):
    diff = cv2.absdiff(background.astype('uint8'),frame)
    ret, thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    
    else:
        # Assuming the Largest external Contour in ROI is  the hand
        hand_segment = max(contours,key=cv2.contourArea)
    
    return (thresholded,hand_segment)


We will use a convex hull by drawing a polygon by connecting points around the most external points in a frame<br><br>
We have to account for the lines from the wrist as our arm is out of the ROI, assuming that we are holding it straight up!
<br><br>
1. We will calculate the most extreme points<br>
top, bottom, left, and right <br>
2.Then we can calculate their intersection and estimate that as the cen ter of the hand
3. Next we will calculate the distance for the point furthest away from the center
4. We will 0.9 of such distance to draw a circle. Any points of the convex hull outside of the circle and far away enough from the bottom should be extended fingers!

In [5]:
def count_fingers(threshold,hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    #TOP
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])

    center_X = (left[0] + right[0]) // 2
    center_Y = (top[1] + bottom[1]) // 2
    # return the distance of all points
    distance = pairwise.euclidean_distances([(center_X,center_Y)], Y = [left, right, top, bottom])[0]
    max_distance = distance.max()
    
    # Create a circle with 90% radius of the max euclidean distance
    radius = int(0.8 * max_distance)
    P = (2 * np.pi * radius)

    # Not grab an ROI of only that circle
    cirROI = np.zeros(threshold.shape[:2], dtype="uint8")
    cv2.circle(cirROI, (center_X, center_Y), radius, 255, 10)
    cirROI = cv2.bitwise_and(threshold, threshold, mask=cirROI)
    image, contours, hierarchy = cv2.findContours(cirROI.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        # To avoid overcalculating points when there is too much of the hand in the ROI
        out_of_wrist = ((center_Y + (center_Y*0.25)) > (y+h))# Avoid piking up below wrist countors
        # Noise contour handle
        limit_points = ((P * 0.25) > contour.shape[0])
        if out_of_wrist and limit_points:
            count += 1
            
    return count

In [6]:
cap = cv2.VideoCapture(0)
num_frames = 0
while True:
    
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    #To not be annoying 
    
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray, accumlated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
    else:
        hand = segment(gray)
        if hand is not None:
            thresholded,hand_segment = hand
            # Draw contours around the true hand frame
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            fingers = count_fingers(thresholded,hand_segment)
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Thresholded',thresholded)
    cv2.rectangle(frame_copy, (roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frames +=1
    cv2.imshow('Finger Count', frame_copy)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()